In [1]:
import numpy as np
import json
import os
from scipy.spatial import distance

from allennlp.common.file_utils import cached_path
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
from scipy.spatial import distance
from sklearn.metrics.cluster import v_measure_score



import sys
sys.path.append('/home/alonj/contrastive')
from nullspace_projection.src.inlp_oop.counterfactual import generate_counterfactuals
import nullspace_projection.src.debias as debias
import torch

task1 = "bios"
task2 = "bios"
train_name = "train"
dev_name = "dev"
test_name="test"

model_path = f"../s3-link/experiments/models/{task1}/roberta-large"
train_encodings_normal = np.load(model_path + f"/encodings/predicted_{task2}_{train_name}_encoded_representations.npy")
# train_labels = np.load(model_path + f"/encodings/predicted_{task}_{train_name}_labels.npy")
train_preds_normal = np.load(model_path + f"/encodings/predicted_{task2}_{train_name}_predictions.npy")
enc_size = train_encodings_normal.shape[1]
print(enc_size)

dev_encodings_normal = np.load(model_path + f"/encodings/predicted_{task2}_{dev_name}_encoded_representations.npy")
# dev_labels = np.load(model_path + f"/encodings/predicted_{task}_{dev_name}_labels.npy")
dev_preds_normal = np.load(model_path + f"/encodings/predicted_{task2}_{dev_name}_predictions.npy")

test_encodings_normal = np.load(model_path + f"/encodings/predicted_{task2}_{test_name}_encoded_representations.npy")
# dev_labels = np.load(model_path + f"/encodings/predicted_{task}_{dev_name}_labels.npy")
test_preds_normal = np.load(model_path + f"/encodings/predicted_{task2}_{test_name}_predictions.npy")

w = np.load(model_path + "/w.npy")  # shape: (num_classes, enc_size)
b = np.load(model_path + "/b.npy")  # shape: (num_classes)
with open(model_path + "/label2index.json", "r") as f:
    label2index = json.load(f)
    index2label = {label2index[k]: k for k in label2index}
    
    

1024


In [2]:
train_path = "../data/bios/train.jsonl"
dev_path = "../data/bios/dev.jsonl"
test_path = "../data/bios/test.jsonl"

with open(cached_path(dev_path)) as f:
    dev_data = [json.loads(line) for line in f if line.strip() if line.strip()]
# print([d['gold_label'] for d in dev_data])
dev_labels = np.array([label2index[d['label']] for d in dev_data])

with open(cached_path(train_path)) as f:
    train_data = [json.loads(line) for line in f if line.strip() if line.strip()]
train_labels = np.array([label2index[d['label']] for d in train_data])

with open(cached_path(test_path)) as f:
    test_data = [json.loads(line) for line in f if line.strip() if line.strip()]
test_labels = np.array([label2index[d['label']] for d in test_data])


In [3]:
dev_data[0]

{'text': 'He is trained and board certified in Emergency Medicine and is a Fellow of the American College of Emergency Physicians. He is a graduate of Brown University School of Medicine and also has a Masters degree in Public Health from Harvard University. Dr. Antosia has extensive medical writing experience with numerous publications in peer-reviewed journals and textbooks, as well as prior international, public health, and disaster-related experience. He has also lectured at local, national, and international forums including England, Ireland and Italy. In addition, he has published numerous abstracts, original articles and authored several book chapters.',
 'original_text': 'Dr. Robert Antosia is an attending physician at Beth Israel Deaconess Medical Center in Boston, Massachusetts and an Assistant Professor at Harvard Medical School. He is trained and board certified in Emergency Medicine and is a Fellow of the American College of Emergency Physicians. He is a graduate of Brown 

In [33]:
index2label

{0: 'professor',
 1: 'physician',
 2: 'attorney',
 3: 'photographer',
 4: 'journalist',
 5: 'nurse',
 6: 'psychologist',
 7: 'teacher',
 8: 'surgeon',
 9: 'dentist',
 10: 'architect',
 11: 'painter',
 12: 'poet',
 13: 'filmmaker',
 14: 'software_engineer',
 15: 'composer',
 16: 'accountant',
 17: 'dietitian',
 18: 'comedian',
 19: 'pastor',
 20: 'chiropractor',
 21: 'yoga_teacher',
 22: 'paralegal',
 23: 'dj',
 24: 'interior_designer',
 25: 'personal_trainer',
 26: 'rapper'}

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import re
from sklearn.metrics.pairwise import linear_kernel
# nltk.download('stopwords')
stopWords = stopwords.words('english')

texts = []
for c in index2label:
    text = ' '.join([' '.join(re.findall('(\\w+)', doc['text'].lower())) for doc in train_data if doc['label'] == index2label[c]])
    texts.append(text)
    
tfidf = TfidfVectorizer(stop_words = stopWords).fit_transform(texts)

cosine_similarities = linear_kernel(tfidf, tfidf)
# cosine_similarities

In [7]:
import numpy as np
import nullspace_projection.src.debias as debias
import pandas as pd

# rootdir = "/home/nlp/jacovia/ilp"  # location in biunlp
rootdir = "/home/alonj/contrastive"
dset = "dev"  # "train" "dev" "test"

# orig_encodings = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_train_encoded_representations.npy")
orig_encodings = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_{dset}_encoded_representations.npy")
orig_labels = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_{dset}_labels.npy")
orig_predictions = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_{dset}_predictions.npy")

classifier_w = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/w.npy")
classifier_b = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/b.npy")

from scipy.spatial import distance
from scipy.stats import entropy
from scipy.special import softmax



def dkl(p,q):
    p = p / p.sum(axis=1).reshape(-1, 1)
    q = q / q.sum(axis=1).reshape(-1, 1)
    return entropy(p,q, axis=1) + entropy(q,p, axis=1)

for fact in index2label:
    drop = []
    print(f"\t\t\t\t\t{index2label[fact]}") # vs {index2label[foil]}")
    for foil in index2label:
        if fact == foil:
            drop.append(-1000)
            continue
            
#         directional_nullspace = debias.get_rowspace_projection((classifier_w[foil].reshape((1, -1))))
#         directional_nullspace = np.eye(directional_nullspace.shape[0]) - directional_nullspace

#         print(f"\t\t\t\t\t{index2label[fact]} vs {index2label[foil]}")
        
        predictions_del = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/encodings/predicted_bios_{dset}_highlight_predictions.npy")
        
#         contrastive_examples = np.any([orig_predictions == fact, orig_predictions == foil], axis=0)
        contrastive_examples = np.any([orig_predictions == fact], axis=0)
#         print(contrastive_examples, contrastive_examples.sum())
#         contrastive_examples = np.any([orig_predictions == fact, predictions_del == foil, orig_labels == foil], axis=0)
#         contrastive_examples = np.all([orig_predictions == fact, predictions_del == foil], axis=0)
#         contrastive_examples = np.any([orig_predictions > -10000], axis=0)
    
        encodings = orig_encodings[contrastive_examples]  # only use examples of the fact/foil
        predictions = orig_predictions[contrastive_examples]
        
        if len(encodings) == 0:
            drop.append(0)
            continue

#         contrastive_rowspace_projection = np.load(f"{rootdir}/s3-link/experiments/models/bios/roberta-large/projections/early/contrastive_{max(fact, foil)}_{min(fact, foil)}.npy")
        contrastive_rowspace_projection = debias.get_rowspace_projection((classifier_w[fact] - classifier_w[foil]).reshape(1, -1))
#         contrastive_rowspace_projection = np.eye(contrastive_rowspace_projection.shape[0]) - contrastive_rowspace_projection

        u = classifier_w[fact] - classifier_w[foil]
        contrastive_rowspace_projection = np.outer(u, u.T) / (u.dot(u.T))
#         print(contrastive_rowspace_projection.shape)
    
        encodings_in_contrastive_rowspace = encodings @ contrastive_rowspace_projection #@ directional_nullspace


        assert (predictions == np.argmax(encodings @ classifier_w.T + classifier_b, axis=1)).all()

        predictions_in_contrastive_space = np.argmax(encodings_in_contrastive_rowspace @ classifier_w.T + classifier_b, axis=1)

        dist9 = sum(predictions_in_contrastive_space == predictions) / len(predictions)
    
        prediction_probabilities = softmax(encodings @ classifier_w.T + classifier_b, axis=1)
        prediction_probabilities_contrastive = softmax(encodings_in_contrastive_rowspace @ classifier_w.T + classifier_b, axis=1)
#         prediction_probabilities_both = softmax(encodings_both @ classifier_w.T + classifier_b, axis=1)
#         prediction_probabilities_del = softmax(encodings_del @ classifier_w.T + classifier_b, axis=1)
        
#         dist5 = prediction_probabilities_contrastive[:, fact] - prediction_probabilities_contrastive[:, foil]
#         dist6 = prediction_probabilities_both[:, fact] - prediction_probabilities_both[:, foil]
#         dist7 = (dist5 - dist6).mean()
        
#         dist8 = dkl(prediction_probabilities_contrastive[:, [fact, foil]],
#                     prediction_probabilities_both[:, [fact, foil]]).mean()
        dist10 = dkl(prediction_probabilities_contrastive,
                    prediction_probabilities).mean()
        
                
#         q = prediction_probabilities_contrastive[:, [fact, foil]]
# #         q = prediction_probabilities_both[:, [fact, foil]]
#         p = prediction_probabilities[:, [fact, foil]]
# #         q = prediction_probabilities_del[:, [fact, foil]]
        
#         p = p / p.sum(axis=1).reshape(-1, 1)
#         q = q / q.sum(axis=1).reshape(-1, 1)
#         dist6 = (p[:, 0] - q[:, 0]).mean()

        drop.append(dist10)
    
    ratios = []
    for c in range(len(index2label)):
        genders = [doc['gender'] for doc in train_data if doc['label'] == index2label[c]]
        ratios.append(sum(g == 'm' for g in genders) / len(genders))

    tab = []
    print(fact)
    for c in range(len(index2label)):
        if c == fact:
            continue
        tab.append({
    #         'foil': index2label[c],
            'foil idx': c,
            'foil': index2label[c],
    #         'collapse': diffs[0][c],
            'measure': drop[c],
            'cos-sim': cosine_similarities[fact,c],
            'm ratio': ratios[c]
        })

    tab = sorted(tab, key=lambda v: -v['measure'])

    display(pd.DataFrame(tab))#, columns=['foil', 'collapse', 'cos-sim'])

    for row1,row2 in zip(tab[0:4],tab[-1:-4:-1]):
        print(f" & {row1['foil']} & {row1['measure']:.3f} & {row2['foil']} & {row2['measure']:.3f}")


					professor
0


,foil idx,foil,measure,cos-sim,m ratio
0,7,teacher,2.383358,0.573837,0.398395
1,6,psychologist,2.153176,0.573063,0.381618
2,1,physician,1.919650,0.352405,0.475674
3,2,attorney,1.474685,0.420885,0.621217
4,4,journalist,1.415809,0.506371,0.506323
5,5,nurse,1.375823,0.249773,0.089235
6,10,architect,1.268899,0.520742,0.769130
7,8,surgeon,1.253804,0.450460,0.848932
8,14,software_engineer,1.203816,0.503450,0.843354
9,12,poet,1.123544,0.343901,0.511597


 & teacher & 2.383 & dj & 0.509
 & psychologist & 2.153 & comedian & 0.522
 & physician & 1.920 & yoga_teacher & 0.542
					physician
1


,foil idx,foil,measure,cos-sim,m ratio
0,8,surgeon,3.867259,0.712523,0.848932
1,0,professor,2.399535,0.352405,0.550123
2,5,nurse,2.026691,0.363892,0.089235
3,20,chiropractor,1.935692,0.393260,0.726729
4,6,psychologist,1.464117,0.405883,0.381618
5,9,dentist,1.448607,0.301674,0.648859
6,17,dietitian,1.315782,0.273999,0.072343
7,2,attorney,1.263550,0.229435,0.621217
8,7,teacher,1.241862,0.251159,0.398395
9,4,journalist,1.206128,0.186806,0.506323


 & surgeon & 3.867 & paralegal & 0.847
 & professor & 2.400 & dj & 0.849
 & nurse & 2.027 & photographer & 0.875
					attorney
2


...

 & architect & 3.540 & composer & 0.869
 & photographer & 2.145 & chiropractor & 1.021
 & journalist & 2.054 & pastor & 1.150
					personal_trainer
25


,foil idx,foil,measure,cos-sim,m ratio
0,7,teacher,2.526296,0.406775,0.398395
1,17,dietitian,2.297440,0.399158,0.072343
2,21,yoga_teacher,2.278942,0.285555,0.156640
3,20,chiropractor,1.941702,0.215426,0.726729
4,5,nurse,1.927119,0.175477,0.089235
5,23,dj,1.701721,0.199404,0.853129
6,2,attorney,1.592953,0.255768,0.621217
7,24,interior_designer,1.486603,0.232667,0.194149
8,4,journalist,1.443431,0.346216,0.506323
9,0,professor,1.426782,0.280896,0.550123


 & teacher & 2.526 & painter & 0.926
 & dietitian & 2.297 & poet & 0.931
 & yoga_teacher & 2.279 & composer & 0.953
					rapper
26


,foil idx,foil,measure,cos-sim,m ratio
0,23,dj,2.891930,0.538838,0.853129
1,12,poet,2.705348,0.243766,0.511597
2,18,comedian,1.931428,0.269224,0.786096
3,15,composer,1.753587,0.462786,0.839506
4,11,painter,1.562380,0.250151,0.549987
5,3,photographer,1.486994,0.276814,0.651584
6,14,software_engineer,1.455682,0.215492,0.843354
7,4,journalist,1.352762,0.322975,0.506323
8,16,accountant,1.350378,0.191041,0.636141
9,8,surgeon,1.345796,0.109061,0.848932


 & dj & 2.892 & dietitian & 0.725
 & poet & 2.705 & yoga_teacher & 0.942
 & comedian & 1.931 & architect & 0.964


In [ ]:
import pandas as pd

ratios = []
for c in range(len(index2label)):
    genders = [doc['gender'] for doc in train_data if doc['label'] == index2label[c]]
    ratios.append(sum(g == 'm' for g in genders) / len(genders))

tab = []
print(fact)
for c in range(len(index2label)):
    if c == fact:
        continue
    tab.append({
#         'foil': index2label[c],
        'foil idx': c,
        'foil': index2label[c],
#         'collapse': diffs[0][c],
        'measure': drop[c],
        'cos-sim': cosine_similarities[fact,c],
        'm ratio': ratios[c]
    })

tab = sorted(tab, key=lambda v: -v['measure'])
    
pd.DataFrame(tab)#, columns=['foil', 'collapse', 'cos-sim'])




In [21]:
from IPython.display import display

In [23]:
from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression

X = np.array([[v['cos-sim']] for v in tab])
Y = np.array([v['measure'] for v in tab])
reg = LinearRegression().fit(X, Y)
print('lexical similarity -> dkl\t\t', reg.score(X, Y))

X = np.array([[v['m ratio']] for v in tab])
# Y = np.array([v['dkl drop'] for v in tab])
reg = LinearRegression().fit(X, Y)
print('m ratio -> dkl\t\t\t\t', reg.score(X, Y))

X = np.array([[v['cos-sim'],v['m ratio']] for v in tab])
# Y = np.array([v['dkl drop'] for v in tab])
reg = LinearRegression().fit(X, Y)
print('lexical similarity + m ratio -> dkl\t', reg.score(X, Y))

lexical similarity -> dkl		 0.5320527099142018
m ratio -> dkl				 0.03071002733385375
lexical similarity + m ratio -> dkl	 0.5633119801734481


In [127]:
import numpy as np
import statsmodels.api as sm

X = np.array([[v['cos-sim']] for v in tab])
X = sm.add_constant(X)
y = np.array([v['measure'] for v in tab])
results = sm.OLS(y, X).fit()
print(results.summary())

X = np.array([[v['m ratio']] for v in tab])
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
print(results.summary())

X = np.array([[v['cos-sim'],v['m ratio']] for v in tab])
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.586
Method:                 Least Squares   F-statistic:                     36.34
Date:                Thu, 28 Jan 2021   Prob (F-statistic):           3.18e-06
Time:                        16:37:25   Log-Likelihood:                 115.61
No. Observations:                  26   AIC:                            -227.2
Df Residuals:                      24   BIC:                            -224.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0029      0.001     -2.114      0.0

In [56]:
diffsnp = np.array(diffs[0])
print([index2label[l] for l in np.argsort(diffsnp)][1:])

['physician', 'professor', 'attorney', 'photographer', 'journalist', 'psychologist', 'teacher', 'surgeon', 'dentist', 'poet', 'architect', 'painter', 'filmmaker', 'software_engineer', 'accountant', 'composer', 'dietitian', 'chiropractor', 'pastor', 'comedian', 'yoga_teacher', 'personal_trainer', 'rapper', 'interior_designer', 'paralegal', 'dj']


In [13]:
diffsnp = np.array(diffs[5])
# print(diffsnp)
# print(index2label[np.argmin(diffsnp)])
print([index2label[l] for l in np.argsort(diffsnp)])

['yoga_teacher', 'nurse', 'personal_trainer', 'poet', 'painter', 'photographer', 'interior_designer', 'attorney', 'paralegal', 'dj', 'dietitian', 'filmmaker', 'journalist', 'teacher', 'rapper', 'comedian', 'pastor', 'composer', 'accountant', 'software_engineer', 'psychologist', 'architect', 'physician', 'chiropractor', 'professor', 'dentist', 'surgeon']


In [10]:
for fact in index2label:
    diffsnp = np.array(diffs[fact])
    # print(diffsnp)
    # print(index2label[np.argmin(diffsnp)])
    print(index2label[fact], '\t\t', [index2label[l] for l in np.argsort(-diffsnp)[:5]])

professor 		 ['physician', 'paralegal', 'attorney', 'nurse', 'dietitian']
physician 		 ['surgeon', 'professor', 'dentist', 'nurse', 'psychologist']
attorney 		 ['paralegal', 'professor', 'architect', 'accountant', 'psychologist']
photographer 		 ['interior_designer', 'paralegal', 'painter', 'filmmaker', 'composer']
journalist 		 ['teacher', 'filmmaker', 'paralegal', 'composer', 'professor']
nurse 		 ['surgeon', 'dentist', 'professor', 'physician', 'chiropractor']
psychologist 		 ['professor', 'chiropractor', 'teacher', 'surgeon', 'nurse']
teacher 		 ['journalist', 'professor', 'filmmaker', 'architect', 'psychologist']
surgeon 		 ['physician', 'nurse', 'chiropractor', 'dietitian', 'dentist']
dentist 		 ['nurse', 'chiropractor', 'physician', 'surgeon', 'dietitian']
architect 		 ['interior_designer', 'paralegal', 'professor', 'nurse', 'attorney']
painter 		 ['photographer', 'filmmaker', 'composer', 'interior_designer', 'journalist']
poet 		 ['composer', 'filmmaker', 'paralegal', 'interior

In [12]:
for fact in index2label:
    diffsnp = np.array(diffs[fact])
    # print(diffsnp)
    # print(index2label[np.argmin(diffsnp)])
    print(index2label[fact], '\t\t', [index2label[l] for l in np.argsort(-diffsnp)[:5]])

professor 		 ['physician', 'paralegal', 'nurse', 'attorney', 'dietitian']
physician 		 ['surgeon', 'professor', 'dentist', 'nurse', 'psychologist']
attorney 		 ['paralegal', 'professor', 'architect', 'psychologist', 'accountant']
photographer 		 ['interior_designer', 'paralegal', 'painter', 'filmmaker', 'rapper']
journalist 		 ['paralegal', 'teacher', 'filmmaker', 'professor', 'composer']
nurse 		 ['surgeon', 'dentist', 'professor', 'chiropractor', 'physician']
psychologist 		 ['professor', 'chiropractor', 'teacher', 'paralegal', 'nurse']
teacher 		 ['journalist', 'professor', 'filmmaker', 'architect', 'psychologist']
surgeon 		 ['physician', 'nurse', 'chiropractor', 'dietitian', 'dentist']
dentist 		 ['nurse', 'chiropractor', 'physician', 'surgeon', 'dietitian']
architect 		 ['interior_designer', 'professor', 'paralegal', 'nurse', 'teacher']
painter 		 ['photographer', 'filmmaker', 'composer', 'paralegal', 'interior_designer']
poet 		 ['composer', 'paralegal', 'interior_designer', 'fi

In [4]:
import pickle
with open("../data/bios/dev.pickle", "rb") as f:
    data = pickle.load(f)

print([ex for ex in data if ex['p'] == 'model'][15:50])
    
yy = 2700
for a in data[yy:yy+20]:
#     if a['p'] == 'attorney':
        print(json.dumps({'text': a['hard_text'], 'label': a['p']}))
#         break

[{'g': 'f', 'p': 'model', 'text': 'Eva Jude is a Adult Model from Ukraine. She was born on January 1, 2000. She is listed on FreeOnes since 2018 and is currently ranked 12450th place. She currently has 5 gallery links in her own FreeOnes section.', 'start': 39, 'hard_text': 'She was born on January 1, 2000. She is listed on FreeOnes since 2018 and is currently ranked 12450th place. She currently has 5 gallery links in her own FreeOnes section.', 'text_without_gender': '_ was born on January 1, 2000. _ is listed on FreeOnes since 2018 and is currently ranked 12450th place. _ currently has 5 gallery links in _ own FreeOnes section.'}, {'g': 'f', 'p': 'model', 'text': 'Pam Kiss is a German model with really nice and big breasts. In 2014 she got a Netstar Award. This photorealistic 3D printed figurine is made with a full-color 3D printer and received an elaborate finishing touch by hand. The figurine is made of full color sandstone It comes in a lavish gift box that includes a Certificate 

NameError: name 'json' is not defined